In [1]:
# Install all the required libraries

!pip install -U -q tiktoken openai chromaDB sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#import necessary libraries
from pathlib import Path
from IPython.core.display import display
import pandas as pd
from operator import itemgetter
import tiktoken
import openai
import chromadb

In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
data_path = '/content/drive/MyDrive/Myntra+Dataset/data.csv'

## Loading the data
We have downloaded the data from Kaggle that contains lots of products with the information. The images in the dataset is not relevant as we are not using them for queries and in the dataset, the URL's are invalid as well.

Also, my Google drive can't take another 4GB of uploads ;)

In [5]:
fashion_data = pd.read_csv(data_path)

In [6]:
fashion_data.head()

,p_id,name,products,price,colour,brand,img,ratingCount,avg_rating,description,p_attributes
0,17048614,Khushal K Women Black Ethnic Motifs Printed Ku...,"Kurta, Palazzos, Dupatta",5099.0,Black,Khushal K,http://assets.myntassets.com/assets/images/170...,4522.0,4.418399,Black printed Kurta with Palazzos with dupatta...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
1,16524740,InWeave Women Orange Solid Kurta with Palazzos...,"Kurta, Palazzos, Floral Print Dupatta",5899.0,Orange,InWeave,http://assets.myntassets.com/assets/images/165...,1081.0,4.119334,Orange solid Kurta with Palazzos with dupatta<...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
2,16331376,Anubhutee Women Navy Blue Ethnic Motifs Embroi...,"Kurta, Trousers, Dupatta",4899.0,Navy Blue,Anubhutee,http://assets.myntassets.com/assets/images/163...,1752.0,4.161530,Navy blue embroidered Kurta with Trousers with...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
3,14709966,Nayo Women Red Floral Printed Kurta With Trous...,"Kurta, Trouser, Dupatta",3699.0,Red,Nayo,http://assets.myntassets.com/assets/images/147...,4113.0,4.088986,Red printed kurta with trouser and dupatta<br>...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
4,11056154,AHIKA Women Black & Green Printed Straight Kurta,Kurta,1350.0,Black,AHIKA,http://assets.myntassets.com/assets/images/110...,21274.0,3.978377,"Black and green printed straight kurta, has a ...","{'Body Shape ID': '424', 'Body or Garment Size..."


In [7]:
# Dropping the image column
fashion_data = fashion_data.drop(['img'],axis=1)

In [8]:
fashion_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14214 entries, 0 to 14213
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   p_id          14214 non-null  int64  
 1   name          14214 non-null  object 
 2   products      14214 non-null  object 
 3   price         14214 non-null  float64
 4   colour        14214 non-null  object 
 5   brand         14214 non-null  object 
 6   ratingCount   6530 non-null   float64
 7   avg_rating    6530 non-null   float64
 8   description   14214 non-null  object 
 9   p_attributes  14214 non-null  object 
dtypes: float64(3), int64(1), object(6)
memory usage: 1.1+ MB


In [9]:
#Data set is huge and we are limited by the OpenAI embedding. Hence shrinking the data for the test model to 1000 rows
fashion_data.count()

p_id            14214
name            14214
products        14214
price           14214
colour          14214
brand           14214
ratingCount      6530
avg_rating       6530
description     14214
p_attributes    14214
dtype: int64

In [10]:
fashion_data = fashion_data[fashion_data['ratingCount'] > 100][:1000]

In [11]:
# Generating the custom Metadata for our dataset to be embedded in the vector database.
fashion_data['Metadata'] = fashion_data.apply(lambda x: {'category': x['products'], 'colour': x['colour'],'avg_rating': x['avg_rating'],'description.': x['description'],'price': x['price']}, axis=1)

In [12]:
fashion_data['Metadata'][0]

{'category': 'Kurta, Palazzos, Dupatta',
 'colour': 'Black',
 'avg_rating': 4.4183989385227775,
 'description.': "Black printed Kurta with Palazzos with dupatta <br> <br> <b> Kurta design:  </b> <ul> <li> Ethnic motifs printed </li> <li> Anarkali shape </li> <li> Regular style </li> <li> Mandarin collar,  three-quarter regular sleeves </li> <li> Calf length with flared hem </li> <li> Viscose rayon machine weave fabric </li> </ul> <br> <b> Palazzos design:  </b> <ul> <li> Printed Palazzos </li> <li> Elasticated waistband </li> <li> Slip-on closure </li> </ul>Dupatta Length 2.43 meters Width:&nbsp;88 cm<br>The model (height 5'8) is wearing a size S100% Rayon<br>Machine wash",
 'price': 5099.0}

In [13]:
fashion_data['name']

0       Khushal K Women Black Ethnic Motifs Printed Ku...
1       InWeave Women Orange Solid Kurta with Palazzos...
2       Anubhutee Women Navy Blue Ethnic Motifs Embroi...
3       Nayo Women Red Floral Printed Kurta With Trous...
4        AHIKA Women Black & Green Printed Straight Kurta
                              ...                        
7347    Vishudh Lime Green & White Geometric Mandarin ...
7348    Anubhutee Black & Gold-Toned Ethnic Motifs Pri...
7349    HERE&NOW Women Beige & Orange Ethnic Motifs Pr...
7350    Nayo Burgundy  White Floral Printed Pure Cotto...
7351    Sangria Green Floral High-Low Pure Cotton Top ...
Name: name, Length: 1000, dtype: object

## Setting up OpenAI and ChromaDB
In this section, we setup openAI and ChromaDB and embed our data into our database.

In [14]:
openai.api_key = '<PUT YOUR OPEN AI KEY HERE'

In [15]:
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

In [16]:
chroma_data_path = '/content/drive/MyDrive/ChromaDB_Data'

In [17]:
import chromadb

In [18]:
client = chromadb.PersistentClient()

In [19]:
model = "text-embedding-ada-002"
embedding_function = OpenAIEmbeddingFunction(api_key=openai.api_key, model_name=model)

In [20]:
# Creating our collection
fashion_collection = client.get_or_create_collection(name='fashion_collection_final', embedding_function=embedding_function)

In [21]:
documents_list = fashion_data['name'].tolist()
metadata_list = fashion_data['Metadata'].tolist()

In [22]:
# Loading our dataset in the ChromaDB
fashion_collection.add(
    documents = documents_list,
    ids = [str(i) for i in range(0, len(documents_list))],
    metadatas = metadata_list
)

In [23]:
fashion_collection.get(
    ids = ['0','1','2'],
    include = ['embeddings', 'documents', 'metadatas']
)

{'ids': ['0', '1', '2'],
 'embeddings': [[-0.03625224903225899,
   0.0053284671157598495,
   0.022554287686944008,
   -0.009436521679162979,
   -0.023781370371580124,
   0.03001014143228531,
   -0.01472497545182705,
   -0.02132720686495304,
   0.000993669149465859,
   -0.02042023278772831,
   0.01635219156742096,
   0.009169765748083591,
   0.012537569738924503,
   0.00956990011036396,
   -0.015031746588647366,
   0.020060112699866295,
   0.017499245703220367,
   -0.027716033160686493,
   0.008396170102059841,
   -0.022207504138350487,
   -0.02187405899167061,
   0.02915651910007,
   0.009409846737980843,
   -0.0005939506809227169,
   -0.01725916564464569,
   0.012831002473831177,
   0.015058421529829502,
   -0.018112787976861,
   -0.0038112879265099764,
   -0.0017622621962800622,
   -0.00914308987557888,
   -0.010603583417832851,
   0.0003353216452524066,
   -0.03113051876425743,
   -0.0214072335511446,
   0.020780354738235474,
   -0.005111727397888899,
   -0.007502535358071327,
   0.

In [24]:
# Creating cache collection for the quick queries
cache_collection = client.get_or_create_collection(name='Fashion_Cache', embedding_function=embedding_function)

In [25]:
cache_collection.peek()

{'ids': [],
 'embeddings': [],
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

## Performing Search
Searching data within our vector dataset

In [64]:
query = input()

I need some recommendations for office going dresses


In [65]:
# Searching in the cache collection
cache_results = cache_collection.query(
    query_texts=query,
    n_results=1
)

In [66]:
cache_results

{'ids': [['I need a full sleeve tshirt']],
 'distances': [[0.42351430439659193]],
 'metadatas': [[{'distances0': '0.3022514581680298',
    'distances1': '0.3103010952472687',
    'distances2': '0.34027668833732605',
    'distances3': '0.3485611081123352',
    'distances4': '0.3496583104133606',
    'distances5': '0.35400405526161194',
    'distances6': '0.3554557263851166',
    'distances7': '0.3559529185295105',
    'distances8': '0.36021897196769714',
    'distances9': '0.36087146401405334',
    'documents0': 'VividArtsy Lavender Sleeveless Fitted Top',
    'documents1': 'QUIERO Rose Top With Cap Sleeves',
    'documents2': 'DressBerry Green Puff Sleeves Cotton Top',
    'documents3': 'DressBerry Red & Orange Floral Puff Sleeve Regular Top',
    'documents4': 'Roadster Teal Green Shirt Style Top',
    'documents5': 'plusS Mauve Shirt Style Top',
    'documents6': 'QUIERO Black & White Embroidered Top',
    'documents7': 'HERE&NOW Purple & Yellow Puff Sleeve Smocked A-Line Top',
    '

In [67]:
# Searching in the main collection
results = fashion_collection.query(
  query_texts = query,
  n_results = 10
)

In [68]:
results

{'ids': [['479',
   '946',
   '912',
   '757',
   '263',
   '441',
   '507',
   '227',
   '974',
   '971']],
 'distances': [[0.35972291231155396,
   0.36034807562828064,
   0.3642035722732544,
   0.3655945062637329,
   0.3729059100151062,
   0.3734891414642334,
   0.3749767243862152,
   0.37540075182914734,
   0.3792303204536438,
   0.38029491901397705]],
 'metadatas': [[{'avg_rating': 4.045454545454546,
    'category': 'Top, Cotton',
    'colour': 'Green',
    'description.': '<ul><li>Green&nbsp;top</li><li>Solid</li><li>Square neck, short, puff sleeves</li><li>Knitted cotton</li></ul>The model (height 5\'8") is wearing a size SCotton <br> Machine wash',
    'price': 1199.0},
   {'avg_rating': 4.295698924731183,
    'category': 'Jumpsuit',
    'colour': 'Rust',
    'description.': "Rust orange&nbsp;solid&nbsp;layered basic jumpsuit, has an off-shoulder, short sleeves, 2 pockets, concealed zip closure<br>Comes with a fabric beltPolyester <br> Machine washThe model (height 5'8) is weari

In [69]:
# Implementing Cache in Semantic Search

# Set a threshold for cache search
threshold = 0.2

ids = []
documents = []
distances = []
metadatas = []
results_df = pd.DataFrame()


# If the distance is greater than the threshold, then return the results from the main collection.

if cache_results['distances'][0] == [] or cache_results['distances'][0][0] > threshold:
      # Query the collection against the user query and return the top 10 results
      results = fashion_collection.query(
        query_texts = query,
        n_results = 10
      )

      # Store the query in cache_collection as document w.r.t to ChromaDB so that it can be embedded and searched against later
      # Store retrieved text, ids, distances and metadatas in cache_collection as metadatas, so that they can be fetched easily if a query indeed matches to a query in cache
      Keys = []
      Values = []

      for key, val in results.items():
        if val is None:
          continue
        for i in range(10):
          Keys.append(str(key)+str(i))
          Values.append(str(val[0][i]))


      cache_collection.add(
          documents= [query],
          ids = [query],
          metadatas = dict(zip(Keys, Values))
      )

      print("Not found in cache. Found in main collection.")

      result_dict = {'Metadatas': results['metadatas'][0], 'Documents': results['documents'][0], 'Distances': results['distances'][0], "IDs":results["ids"][0]}
      results_df = pd.DataFrame.from_dict(result_dict)
      results_df


# If the distance is, however, less than the threshold, you can return the results from cache

elif cache_results['distances'][0][0] <= threshold:
      cache_result_dict = cache_results['metadatas'][0][0]

      # Loop through each inner list and then through the dictionary
      for key, value in cache_result_dict.items():
          if 'ids' in key:
              ids.append(value)
          elif 'documents' in key:
              documents.append(value)
          elif 'distances' in key:
              distances.append(value)
          elif 'metadatas' in key:
              metadatas.append(value)

      print("Found in cache!")

      # Create a DataFrame
      results_df = pd.DataFrame({
        'IDs': ids,
        'Documents': documents,
        'Distances': distances,
        'Metadatas': metadatas
      })


Not found in cache. Found in main collection.


In [70]:
results_df

,Metadatas,Documents,Distances,IDs
0,"{'avg_rating': 4.045454545454546, 'category': ...",DressBerry Green Puff Sleeves Cotton Top,0.359723,479
1,"{'avg_rating': 4.295698924731183, 'category': ...",DressBerry Women Rust Orange Solid Off-Shoulde...,0.360348,946
2,"{'avg_rating': 4.124113475177305, 'category': ...",DressBerry Women Magenta & White Floral Printe...,0.364204,912
3,"{'avg_rating': 4.057251908396947, 'category': ...",DressBerry Black & Off White Floral Printed Mi...,0.365595,757
4,"{'avg_rating': 4.299019607843137, 'category': ...",Kvsfab Women Attractive Peach Georgette Sequin...,0.372906,263
5,"{'avg_rating': 4.21860465116279, 'category': '...",DressBerry Women White & Black Striped Pure Co...,0.373489,441
6,"{'avg_rating': 4.328, 'category': 'Top, Regula...",DressBerry Red & Orange Floral Puff Sleeve Reg...,0.374977,507
7,"{'avg_rating': 4.373443983402489, 'category': ...",Kvsfab Women Bright Mustard Yellow Georgette S...,0.375401,227
8,"{'avg_rating': 4.487804878048781, 'category': ...",Berrylush Women Navy Blue Polka Dotted Co-Ord ...,0.379230,974
9,"{'avg_rating': 4.457446808510638, 'category': ...",Berrylush Yellow Floral Printed Two-Piece Dress,0.380295,971


In [33]:
from sentence_transformers import CrossEncoder, util

In [34]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [35]:
# Test the cross encoder model

scores = cross_encoder.predict([['I need some shirts along with jeans', 'Help me with a shirt please'],
                                ['White shirt', 'Help me with a white shirt'],
                                ['What is the best pants for office going', 'Give me best rated shirt for a party']])

In [36]:
scores

array([ -5.8979053,   4.7453246, -10.623296 ], dtype=float32)

In [71]:
cross_inputs = [[query, response] for response in results_df['Documents']]
cross_rerank_scores = cross_encoder.predict(cross_inputs)

In [72]:
cross_rerank_scores

array([ -9.9735775, -10.9205265, -10.723936 ,  -9.406349 , -10.425999 ,
        -9.827771 ,  -9.728397 , -10.754864 ,  -9.465578 ,  -9.444928 ],
      dtype=float32)

In [73]:
results_df['Reranked_scores'] = cross_rerank_scores

In [74]:
results_df

,Metadatas,Documents,Distances,IDs,Reranked_scores
0,"{'avg_rating': 4.045454545454546, 'category': ...",DressBerry Green Puff Sleeves Cotton Top,0.359723,479,-9.973577
1,"{'avg_rating': 4.295698924731183, 'category': ...",DressBerry Women Rust Orange Solid Off-Shoulde...,0.360348,946,-10.920527
2,"{'avg_rating': 4.124113475177305, 'category': ...",DressBerry Women Magenta & White Floral Printe...,0.364204,912,-10.723936
3,"{'avg_rating': 4.057251908396947, 'category': ...",DressBerry Black & Off White Floral Printed Mi...,0.365595,757,-9.406349
4,"{'avg_rating': 4.299019607843137, 'category': ...",Kvsfab Women Attractive Peach Georgette Sequin...,0.372906,263,-10.425999
5,"{'avg_rating': 4.21860465116279, 'category': '...",DressBerry Women White & Black Striped Pure Co...,0.373489,441,-9.827771
6,"{'avg_rating': 4.328, 'category': 'Top, Regula...",DressBerry Red & Orange Floral Puff Sleeve Reg...,0.374977,507,-9.728397
7,"{'avg_rating': 4.373443983402489, 'category': ...",Kvsfab Women Bright Mustard Yellow Georgette S...,0.375401,227,-10.754864
8,"{'avg_rating': 4.487804878048781, 'category': ...",Berrylush Women Navy Blue Polka Dotted Co-Ord ...,0.379230,974,-9.465578
9,"{'avg_rating': 4.457446808510638, 'category': ...",Berrylush Yellow Floral Printed Two-Piece Dress,0.380295,971,-9.444928


In [75]:
top_5_semantic = results_df.sort_values(by='Distances')
top_5_semantic[:5]

,Metadatas,Documents,Distances,IDs,Reranked_scores
0,"{'avg_rating': 4.045454545454546, 'category': ...",DressBerry Green Puff Sleeves Cotton Top,0.359723,479,-9.973577
1,"{'avg_rating': 4.295698924731183, 'category': ...",DressBerry Women Rust Orange Solid Off-Shoulde...,0.360348,946,-10.920527
2,"{'avg_rating': 4.124113475177305, 'category': ...",DressBerry Women Magenta & White Floral Printe...,0.364204,912,-10.723936
3,"{'avg_rating': 4.057251908396947, 'category': ...",DressBerry Black & Off White Floral Printed Mi...,0.365595,757,-9.406349
4,"{'avg_rating': 4.299019607843137, 'category': ...",Kvsfab Women Attractive Peach Georgette Sequin...,0.372906,263,-10.425999


In [76]:
top_5_rerank = results_df.sort_values(by='Reranked_scores', ascending=False)
top_5_rerank[:5]

,Metadatas,Documents,Distances,IDs,Reranked_scores
3,"{'avg_rating': 4.057251908396947, 'category': ...",DressBerry Black & Off White Floral Printed Mi...,0.365595,757,-9.406349
9,"{'avg_rating': 4.457446808510638, 'category': ...",Berrylush Yellow Floral Printed Two-Piece Dress,0.380295,971,-9.444928
8,"{'avg_rating': 4.487804878048781, 'category': ...",Berrylush Women Navy Blue Polka Dotted Co-Ord ...,0.379230,974,-9.465578
6,"{'avg_rating': 4.328, 'category': 'Top, Regula...",DressBerry Red & Orange Floral Puff Sleeve Reg...,0.374977,507,-9.728397
5,"{'avg_rating': 4.21860465116279, 'category': '...",DressBerry Women White & Black Striped Pure Co...,0.373489,441,-9.827771


In [77]:
top_5_RAG = top_5_rerank[["Documents", "Metadatas"]][:5]

In [78]:
top_5_RAG

,Documents,Metadatas
3,DressBerry Black & Off White Floral Printed Mi...,"{'avg_rating': 4.057251908396947, 'category': ..."
9,Berrylush Yellow Floral Printed Two-Piece Dress,"{'avg_rating': 4.457446808510638, 'category': ..."
8,Berrylush Women Navy Blue Polka Dotted Co-Ord ...,"{'avg_rating': 4.487804878048781, 'category': ..."
6,DressBerry Red & Orange Floral Puff Sleeve Reg...,"{'avg_rating': 4.328, 'category': 'Top, Regula..."
5,DressBerry Women White & Black Striped Pure Co...,"{'avg_rating': 4.21860465116279, 'category': '..."


In [79]:
def generate_response(query, results_df):
    """
    Generate a response using GPT-3.5's ChatCompletion based on the user query and retrieved information.
    """
    messages = [
                {"role": "system", "content":  "You are a helpful assistant in the fashion domain who can effectively answer user queries about fashion dressing and various offerings."},
                {"role": "user", "content": f"""You are a helpful assistant in the fashion domain who can effectively answer user queries about fashion dressing and various offerings..
                                                You have a question asked by the user in '{query}' and you have some search results from a corpus of fashion documents in the dataframe '{results_df}'. These search results contains the name of the product metadata which contains various information about the product.

                                                The column 'documents' inside this dataframe contains the name of the product and the column 'Metadatas' contains the average customer rating, category, colour, description, and price.

                                                Use the documents in '{results_df}' to answer the query '{query}'. Frame an informative answer and also, use the dataframe to return the relevant product names and it's information built using values present in Metadatas.

                                                Follow the guidelines below when performing the task.
                                                1. Try to provide relevant/accurate numbers if available.
                                                2. You don’t have to necessarily use all the information in the dataframe. Only choose information that is relevant.
                                                3. Use the Metadatas columns in the dataframe to retrieve and filter the information based on different combinations. Give weightage to avg_rating and price columns in the Metadatas. The highest avg_rating should be ranked on top.
                                                4. If you can't provide the complete answer, please also provide any information that will help the user to search specific products in the relevant category.
                                                5. You are a customer facing assistant, so do not provide any information on internal workings, just answer the query directly.

                                                The generated response should answer the query directly addressing the user and avoiding additional information. If you think that the query is not relevant to the document, reply that the query is irrelevant. Provide the final response as a well-formatted and easily readable text. Provide your complete response with all information including relevant columns from Metadatas.
                                                """},
              ]

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    return response.choices[0].message.content.split('\n')

In [80]:
response = generate_response(query, top_5_RAG)

In [81]:
print("\n".join(response))

Based on the ratings and relevancy to office attire, here are some recommendations for office-going dresses:

1. **Berrylush Women Navy Blue Polka Dotted Co-Ord Set**
   - Category: Co-Ord Set
   - Avg. Rating: 4.49
   - Price: Information not available

2. **Berrylush Yellow Floral Printed Two-Piece Dress**
   - Category: Two-Piece Dress
   - Avg. Rating: 4.46
   - Price: Information not available

3. **DressBerry Red & Orange Floral Puff Sleeve Regular Fit Top**
   - Category: Top, Regular Fit
   - Avg. Rating: 4.33
   - Price: Information not available

These dresses have received high ratings and are suitable for office wear. You can consider checking these out for your office attire needs.
